<a href="https://colab.research.google.com/github/yuki-2000/Image_processing_lecture/blob/main/IP2022_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 初期設定（このセルの中身は触らなくて良い）
import numpy as np # 数値解析パッケージnumpyのimport
import cv2 # 画像処理パッケージopencvのimport
import matplotlib.pyplot as plt # 描画パッケージmatplotlibのimport
import imutils

# Google Colab固有の設定
from google.colab import files
from google.colab.patches import cv2_imshow

# Google driveのmount
from google.colab import drive
drive.mount('/content/drive/')
# Google Colab用directoryに移動
%cd "/content/drive/MyDrive/Colab Notebooks/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [ ]:
### 6-1：オプティカルフロー

# 動画像の読込
video = cv2.VideoCapture('https://bit.ly/3V63Pny')

# 読み込めなかった場合の例外処理
if not video.isOpened():
  print ("File Cannot be Opened")

# 色の生成
color = np.random.randint(low=0,high=255,size=(100,3))
# 動画像の先頭のフレームを読み込む
ret, old_frame = video.read()
# 濃淡画像に変換
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
# Shi-Tomasiコーナ点検出を適用
old_corners = cv2.goodFeaturesToTrack(old_gray, maxCorners=100,qualityLevel=0.01, minDistance=10)
# 検出結果を記録する黒い画像を用意
mask = np.zeros_like(old_frame)
# 以降はループで1フレームずつ処理
while(1):

  # 動画像から次のフレームを読み込む
  ret, new_frame = video.read()

  # フレームを読み込めなかった場合（動画の最後まで到達）中断
  if not(ret):
    break


  # 濃淡画像に変換
  new_gray = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)



  # オプティカルフローの計算
  new_corners, st, err = cv2.calcOpticalFlowPyrLK(old_gray, # 旧フレーム画像
                                                    new_gray,  # 新フレーム画像
                                                    old_corners,  # 旧コーナ点
                                                    None,  # 新コーナ点
                                                    winSize=(15,15),  # 探索範囲
                                                    maxLevel=2,  # ピラミッド画像の段数
                                                    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))  # 終了判定

  # 新旧フレームで対応付くコーナ点の選択
  good_new = new_corners[st==1]
  good_old = old_corners[st==1]
    # 各対応コーナ点間のフローの描画
  for i, (new, old) in enumerate(zip(good_new, good_old)):
    a,b = new.ravel()
    c,d = old.ravel()
        
      # マスク画像上で新旧の対応点間を線分で結ぶ
    mask = cv2.line(mask, pt1=(int(a), int(b)), pt2 = (int(c), int(d)), color = color[i].tolist(),thickness=2)

      # 新フレーム上のコーナ点を円で表示
    new_frame = cv2.circle(new_frame, center=(int(a), int(b)), radius=5, color=color[i].tolist(),thickness =-1)

  # マスク画像を新フレーム画像に重ねて表示
  img = cv2.add(new_frame, mask)
  cv2_imshow(img)








  # 旧フレームと旧コーナ点を更新
  old_gray = new_gray.copy()
  old_corners = good_new.reshape(-1,1,2)

# 動画像ファイルを解放
video.release()